<a href="https://colab.research.google.com/github/antonysama/exploratory/blob/master/GasRFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install pkgs
# install.packages(c('readxl', 'lubridate', 'psych', dependencies = TRUE, repos = "https://cran.rstudio.com/"))
x<-c('readxl', 'lubridate', 'dplyr') #'psych'
lapply(x, require, character.only = TRUE)

Loading required package: readxl

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

In [17]:
#For the scenario mode, subset fiscal years and aggregate the sum for C5 production and mean for C5 spec price
df<-read_excel('Gas_RFM.xlsx', sheet = "Calculation Monthly",skip =6  ) %>%
  subset(Fiscal_Year == "2022 / 2023" | Fiscal_Year == "2023 / 2024") %>%
  rename(C5RefPriceSpec_CAD_M3 = 'C5RefPriceSpec_CAD/M3') %>%
  group_by(Fiscal_Year) %>%
  summarize(across(ProdC5TotalAll_000M3, sum), across(C5RefPriceSpec_CAD_M3, mean))
df

Fiscal_Year,ProdC5TotalAll_000M3,C5RefPriceSpec_CAD_M3
<chr>,<dbl>,<dbl>
2022 / 2023,15228.73,719.8525
2023 / 2024,14366.04,594.7277


In [18]:
#Do the same thing as above for Budget Gas RFM model
dfb<-read_excel('BudgetGas_RFM.xlsx', sheet = "Calculation Monthly",skip =6  ) %>%
  subset(Fiscal_Year == "2022 / 2023" | Fiscal_Year == "2023 / 2024") %>%
  rename(C5RefPriceSpec_CAD_M3 = 'C5RefPriceSpec_CAD/M3') %>%
  group_by(Fiscal_Year) %>%
  summarize(across(ProdC5TotalAll_000M3, sum), across(C5RefPriceSpec_CAD_M3, mean))
dfb

Fiscal_Year,ProdC5TotalAll_000M3,C5RefPriceSpec_CAD_M3
<chr>,<dbl>,<dbl>
2022 / 2023,15424.40,720.0280
2023 / 2024,16337.27,596.4336


In [22]:
#Subtract df-dfb, by first join the two tibbles by the common Fiscal_Year column
left_join(df, dfb, by = "Fiscal_Year") %>%
  mutate(difference_ProdC5TotalAll_000M3 = ProdC5TotalAll_000M3.x - ProdC5TotalAll_000M3.y,
         difference_C5RefPriceSpec_CAD_M3 = C5RefPriceSpec_CAD_M3.x - C5RefPriceSpec_CAD_M3.y)

Fiscal_Year,ProdC5TotalAll_000M3.x,C5RefPriceSpec_CAD_M3.x,ProdC5TotalAll_000M3.y,C5RefPriceSpec_CAD_M3.y,difference_ProdC5TotalAll_000M3,difference_C5RefPriceSpec_CAD_M3
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022 / 2023,15228.73,719.8525,15424.40,720.0280,-195.6704,-0.1754772
2023 / 2024,14366.04,594.7277,16337.27,596.4336,-1971.2317,-1.7058965


In [ ]:
# (Optionsl) Inpect data quality & types  ---- this was TOO BIG to inspect, but the pkg runs faster than skimr
# describe(df) # pckg psych

In [ ]:
#(OPTIONAL), create a new fscl_yr column, a numeric one as opposed to above chr (Optional)
df$fscl_yr <- ifelse ( month(df$Date)<4, year(df$Date)-1, year(df$Date) )

#Inspect both fscl year columns
head(df[, c("fscl_yr", "Fiscal_Year")])

fscl_yr,Fiscal_Year
<dbl>,<chr>
2017,2017 / 2018
2017,2017 / 2018
2017,2017 / 2018
2018,2018 / 2019
2018,2018 / 2019
2018,2018 / 2019
